In [1]:
!pip install openai==0.28.1
!pip install fpdf==1.7.2

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [ ]:
import pandas as pd
import openai
import time
from fpdf import FPDF


In [ ]:
openai.api_key = ''

In [5]:
df = pd.read_csv('https://raw.githubusercontent.com/alpharomercoma/AI-Republic-Bootcamp/refs/heads/master/day_4/parcel_data.csv')

In [6]:
df.sample(5)

,ParcelID,SenderName,SenderAddress,SenderCity,SenderCountry,ReceiverName,ReceiverAddress,ReceiverCity,ReceiverCountry,ParcelWeight,ParcelDimensions,ShipmentDate,EstimatedDeliveryDate,CurrentStatus,DeliveryService,TrackingURL
5,PCL1005,Emma Brown,303 Elm Ct,Philadelphia,USA,Mason Walker,304 Elm Ct,Las Vegas,USA,18.93,94x25x62,2024-10-19,2024-11-18,Out for Delivery,UPS,https://tracking.example.com/1005
7,PCL1007,Grace Young,505 Willow Way,San Diego,USA,Logan Wilson,506 Willow Way,Baltimore,USA,9.32,28x30x32,2024-03-14,2024-12-23,Out for Delivery,FedEx,https://tracking.example.com/1007
17,PCL1017,Rachel Lewis,666 Hickory St,Seattle,USA,Benjamin Hall,686 Hickory St,Raleigh,USA,17.70,20x93x49,2024-09-15,2024-11-28,In Transit,USPS,https://tracking.example.com/1017
14,PCL1014,Nancy Wright,333 Redwood Ct,San Francisco,USA,Isabella Wright,353 Redwood Ct,Mesa,USA,16.28,42x51x13,2024-07-14,2024-12-28,Delivered,DHL,https://tracking.example.com/1014
4,PCL1004,Chris King,202 Birch Blvd,Phoenix,USA,Sophia White,203 Birch Blvd,Nashville,USA,8.11,14x55x56,2024-07-06,2024-11-08,Delivered,DHL,https://tracking.example.com/1004


In [7]:
# Get Column Names and Data Types
column_info = df.dtypes
print(column_info)

ParcelID                  object
SenderName                object
SenderAddress             object
SenderCity                object
SenderCountry             object
ReceiverName              object
ReceiverAddress           object
ReceiverCity              object
ReceiverCountry           object
ParcelWeight             float64
ParcelDimensions          object
ShipmentDate              object
EstimatedDeliveryDate     object
CurrentStatus             object
DeliveryService           object
TrackingURL               object
dtype: object


In [8]:
# Identify Categorical and Numerical Features
categorical_features = df.select_dtypes(include=['object']).columns
numerical_features = df.select_dtypes(include=['int64', 'float64']).columns

print("categorical_features:", categorical_features)
print("numerical_features:", numerical_features)

categorical_features: Index(['ParcelID', 'SenderName', 'SenderAddress', 'SenderCity',
       'SenderCountry', 'ReceiverName', 'ReceiverAddress', 'ReceiverCity',
       'ReceiverCountry', 'ParcelDimensions', 'ShipmentDate',
       'EstimatedDeliveryDate', 'CurrentStatus', 'DeliveryService',
       'TrackingURL'],
      dtype='object')
numerical_features: Index(['ParcelWeight'], dtype='object')


In [10]:
# Get Unique Values for Categorical Features with Limitation
def get_unique_values(col, max_display =10):
    uniques = df[col].unique().tolist()
    if len(uniques) > max_display:
        return uniques[:max_display] + ['...']
    return uniques

unique_values = {col: get_unique_values(col) for col in categorical_features}


print("Unique Values for Categorical Features:")
for col, uniques in unique_values.items():
    print(f" - {col}: {uniques}")
print()

Unique Values for Categorical Features:
 - ParcelID: ['PCL1000', 'PCL1001', 'PCL1002', 'PCL1003', 'PCL1004', 'PCL1005', 'PCL1006', 'PCL1007', 'PCL1008', 'PCL1009', '...']
 - SenderName: ['John Doe', 'Jane Smith', 'Alice Johnson', 'Bob Lee', 'Chris King', 'Emma Brown', 'Frank White', 'Grace Young', 'Henry Hall', 'Isabel Clark', '...']
 - SenderAddress: ['123 Maple St', '456 Oak Ave', '789 Pine Rd', '101 Cedar Ln', '202 Birch Blvd', '303 Elm Ct', '404 Walnut Dr', '505 Willow Way', '606 Ash Pkwy', '707 Cherry Pl', '...']
 - SenderCity: ['New York', 'Los Angeles', 'Chicago', 'Houston', 'Phoenix', 'Philadelphia', 'San Antonio', 'San Diego', 'Dallas', 'San Jose', '...']
 - SenderCountry: ['USA']
 - ReceiverName: ['Mia Lopez', 'Noah Carter', 'Olivia King', 'Liam Brown', 'Sophia White', 'Mason Walker', 'Amelia Johnson', 'Logan Wilson', 'Ava Moore', 'Ethan Thomas', '...']
 - ReceiverAddress: ['234 Maple St', '567 Oak Ave', '890 Pine Rd', '102 Cedar Ln', '203 Birch Blvd', '304 Elm Ct', '405 Waln

In [11]:
# Get Descriptive Statistics for Numerical Features
numerical_stats = df[numerical_features].describe()
print("Descriptive Statistics for Numerical Features:\n", numerical_stats, "\n")

Descriptive Statistics for Numerical Features:
        ParcelWeight
count     20.000000
mean      14.844000
std        7.226027
min        1.060000
25%        9.762500
50%       15.205000
75%       21.420000
max       24.270000 



In [12]:
def generate_template(df,
                      column_info,
                      categorical_features,
                      numerical_features,
                      unique_values,
                      numerical_stats):
    # Construct a summary of the dataframe's structure
    column_summary = "Column Names and Data Types:\n"
    for col, dtype in column_info.items():
        column_summary += f" - {col}: {dtype}\n"

    # Unique values for categorical features
    unique_values_str = "Unique Values for Categorical Features:\n"
    for col, uniques in unique_values.items():
        unique_values_str += f" - {col}: {uniques}\n"

    # Descriptive statistics for numerical features
    numerical_stats_str = "Descriptive Statistics for Numerical Features:\n"
    for col in numerical_features:
        numerical_stats_str += f" - {col}:\n"
        for stat_name, value in numerical_stats[col].items():
            numerical_stats_str += f"   - {stat_name}: {value}\n"

    # Define the system prompt
    system_prompt = """
    You are an intelligent assistant that creates descriptive templates for transforming dataframe rows into coherent paragraphs.
    Analyze the provided dataframe structure and generate a template sentence that includes placeholders for each column.
    Ensure the template is contextually relevant and maintains grammatical correctness.
    """

    # Define the user prompt
    user_prompt = f"""
    Analyze the following dataframe structure and create a descriptive template with placeholders for each column.

    <column_summary>
    {column_summary}
    </column_summary>

    <unique_values>
    {unique_values_str}
    </unique_values>

    <numerical_stats>
    {numerical_stats_str}
    </numerical_stats>

    Use the exact column names from the column_summary in generating the variable names in the template,
    as they will be populated with the actual values in the dataset.

    Example Template about a Spotify dataset:
    "{{artist}} gained {{streams}} streams in the song '{{song}}' that was a hit in {{date}}."

    Output only the template without any explanation or introduction.
    The template's variables will be dynamically replaced so make sure they're formatted properly.
    """

    # Generate the template (with retries)
    retries = 3
    for attempt in range(retries):
        try:
            response = openai.ChatCompletion.create(
                model="gpt-4o-mini",
                temperature=0.3,
                max_tokens=1024,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ]
            )
            template = response['choices'][0]['message']['content'].strip()
            return template
        except Exception as e:
            print(f"Error generating template (Attempt {attempt + 1}/{retries}): {e}")
            time.sleep(2)  # Wait before retrying

    return None

In [13]:
# Generate the template
template = generate_template(df,
                             column_info,
                             categorical_features,
                             numerical_features,
                             unique_values,
                             numerical_stats)
print(template)

"Parcel {ParcelID}, sent by {SenderName} from {SenderAddress}, {SenderCity}, {SenderCountry}, is currently {CurrentStatus}. It is addressed to {ReceiverName} at {ReceiverAddress}, {ReceiverCity}, {ReceiverCountry}. The parcel weighs {ParcelWeight} kg and has dimensions of {ParcelDimensions}. It was shipped on {ShipmentDate} and is estimated to be delivered by {EstimatedDeliveryDate} via {DeliveryService}. You can track it using the following link: {TrackingURL}."


In [14]:
def populate_template(template, row):

    # Convert row to dictionary and replace NaN with 'N/A'
    row_dict = row.to_dict()
    for key, value in row_dict.items():
        if pd.isna(value):
            row_dict[key] = 'N/A'
    paragraph = template.format(**row_dict)
    return paragraph

    # Generate the populated template per row
    paragraph = template.format(**row_dict)
    return paragraph


In [15]:
# Apply the populate_template function to each row in the dataframe
df['paragraph'] = df.apply(lambda row: populate_template(template, row), axis=1)


In [16]:
df['paragraph'][0]

'"Parcel PCL1000, sent by John Doe from 123 Maple St, New York, USA, is currently Processing. It is addressed to Mia Lopez at 234 Maple St, Miami, USA. The parcel weighs 24.27 kg and has dimensions of 12x97x48. It was shipped on 2024-02-25 and is estimated to be delivered by 2024-12-07 via UPS. You can track it using the following link: https://tracking.example.com/1000."'

In [17]:
df.sample(5)

,ParcelID,SenderName,SenderAddress,SenderCity,SenderCountry,ReceiverName,ReceiverAddress,ReceiverCity,ReceiverCountry,ParcelWeight,ParcelDimensions,ShipmentDate,EstimatedDeliveryDate,CurrentStatus,DeliveryService,TrackingURL,paragraph
14,PCL1014,Nancy Wright,333 Redwood Ct,San Francisco,USA,Isabella Wright,353 Redwood Ct,Mesa,USA,16.28,42x51x13,2024-07-14,2024-12-28,Delivered,DHL,https://tracking.example.com/1014,"""Parcel PCL1014, sent by Nancy Wright from 333..."
2,PCL1002,Alice Johnson,789 Pine Rd,Chicago,USA,Olivia King,890 Pine Rd,Atlanta,USA,20.99,21x25x45,2024-10-17,2024-11-19,Processing,FedEx,https://tracking.example.com/1002,"""Parcel PCL1002, sent by Alice Johnson from 78..."
3,PCL1003,Bob Lee,101 Cedar Ln,Houston,USA,Liam Brown,102 Cedar Ln,Boston,USA,24.07,89x36x71,2024-03-06,2024-12-09,In Transit,FedEx,https://tracking.example.com/1003,"""Parcel PCL1003, sent by Bob Lee from 101 Ceda..."
4,PCL1004,Chris King,202 Birch Blvd,Phoenix,USA,Sophia White,203 Birch Blvd,Nashville,USA,8.11,14x55x56,2024-07-06,2024-11-08,Delivered,DHL,https://tracking.example.com/1004,"""Parcel PCL1004, sent by Chris King from 202 B..."
5,PCL1005,Emma Brown,303 Elm Ct,Philadelphia,USA,Mason Walker,304 Elm Ct,Las Vegas,USA,18.93,94x25x62,2024-10-19,2024-11-18,Out for Delivery,UPS,https://tracking.example.com/1005,"""Parcel PCL1005, sent by Emma Brown from 303 E..."


In [18]:
def save_content_to_txt(paragraphs, txt_filename):
    try:
        with open(txt_filename, 'w', encoding='utf-8') as file:

            for para in paragraphs:
                file.write(para + '\n\n')  # Double newline for separation

        print(f"All content has been saved as '{txt_filename}'.\n")
    except Exception as e:
        print(f"Error saving content to text file: {e}")

In [19]:
def save_content_to_pdf(paragraphs, pdf_filename):
    try:
        pdf = FPDF()
        pdf.add_page()
        pdf.set_auto_page_break(auto=True, margin=15)
        pdf.set_font("Arial", size=12)

        # Add each paragraph
        for para in paragraphs:
            pdf.multi_cell(0, 10, para)
            pdf.ln()  # Add a line break between paragraphs

        pdf.output(pdf_filename)
        print(f"All content has been saved as '{pdf_filename}'.\n")
    except Exception as e:
        print(f"Error saving content to PDF: {e}")

In [20]:
# Save the paragraphs to both text and PDF files
save_content_to_txt(df['paragraph'].to_list(),
                    txt_filename='nlg.txt')

save_content_to_pdf(df['paragraph'].to_list(),
                    pdf_filename='nlg.pdf')

All content has been saved as 'nlg.txt'.

All content has been saved as 'nlg.pdf'.

